<a href="https://colab.research.google.com/github/lidar532/ppkgeotag/blob/master/PPK_2_PixPos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPK-2-PixPos

## About PPK-2-PixPos
---
By: C. W. Wright<br/>
wright(AT)[lidar.net](https://lidar.net)

***PPK-2-PixPos*** software is designed to do the following:
1. Run completely online from a [Google Chrome](https://www.google.com/chrome/), [FireFox](https://www.mozilla.org/en-US/), or [Safari](https://www.apple.com/safari/) web browser
1. Process raw GNSS dual frequency carrier phase generated by the [CWW-PPK](http://lidar.net) Precision GNSS system to precision trajectories.
1. Create a unified directory structure for a GPS and SfM project,
1.  Analyse and compare 
  * GNSS trajectories, 
  * Photo event time Synchronization 
1. Use a GNSS trajectory to process Photo Events from the [CWW-PPK](http://lidar.net) into precision photo positions that can be loaded loaded into Agisoft Metashape for SfM processing.

### Leveraged Software:
* [RTKlib:](http://www.rtklib.com/) An Open Source Program Package for GNSS Positioning. The RTKlib manual in pdf can be found at: [www.rtklib.com](http://www.rtklib.com/prog/manual_2.4.2.pdf).  ***PPK-2-PixPos*** uses the Linux commandline version of the RTKlib postprocessor (RNX2RTKP) and other tools.
* [Teqc: ](https://www.unavco.org/software/data-processing/teqc/teqc.html) The Toolkit for GNSS Data.


----
### Example datasets
 Various example datasets [can be found here](https://drive.google.com/open?id=1YjjvH3uTRHRt06CHT6b1NvlgZMmnsJqo).  Download a dataset to your computer, 
and unzip it.  Create a new project using A.0.2 below.  Upload the various gps files to the appropriate subdirectories in the new project.  **DO NOT UPLOAD THE ACTUAL PHOTOS** as they are not required and would take consideralbe time to upload and disk space to store.





# 0.0.0.0 EXIF Extractor for MetaShape / PhotoScan

## A.0.0.0 EXIF extractor for Metashape / Photoscan


Cut and paste the following code to a file on your computer named:  **'extractEXIF.py'**.  Load your photos into ***MetaShape*** and then run **'extractEXIF.py'** to generate a file containing the necessary EXIF information fo the PPK system. 

```
# Script to write select exif info from within PhotoScan
# Run from Tools->Run Script
# Christine Kranenburg
# 2018-03-22

import PhotoScan
doc = PhotoScan.app.document
chunk = doc.chunk
path = PhotoScan.app.getSaveFileName()
file = open(path, "w")

for i in range(0, len(chunk.cameras)):
	camera = chunk.cameras[i]
	print("camera.photo.meta:",camera.photo.meta)
	dt = camera.photo.meta['Exif/DateTimeOriginal']
	ss = "1/" + str(int(1/float(camera.photo.meta["Exif/ExposureTime"])))
	iso = camera.photo.meta["Exif/ISOSpeedRatings"]
	bits = [camera.label, dt, iso, ss]
	print("Bits:",bits)
#	strout = ",".join(bits)
	print(camera.label,',',dt,',',iso,',',ss)
	my_str = camera.label+','+dt+','+iso+','+ss
#	file.write(strout + "\n")
	file.write(my_str+'\n')
	print(camera.label, dt, iso, ss)
	
file.close()
print("all done");

```



# **A.** Convert & Process Flight GNSS with RTKlib.

## A.0 Start here.

In [0]:
#@title A.0.1 **(Required First Step)** Download and install RTKlib from the lidar532 github repository {form-width:"25%"}

import pandas as pd
import os
import pathlib
import numpy as np
from   bokeh.plotting import figure, show, output_file
from   bokeh.layouts import gridplot
from   bokeh.io import output_notebook
import ipywidgets as widgets
from   IPython.display import display
from   google.colab import widgets

# Function to extract a single variable from a file.
def get_file_var( fn, var ):
  for n, line in enumerate(open(fn)):
    if var in line:
      return line

def init_analysis():
  global settings,  trajectories, Trj, binSize_In_cm, binsz,c, Trj_fn
  global Trj1, Trj2, Trj3, Trj4, Trj5, Trj6, Trj7

  try:
    settings
    trajectories
  except NameError:
    trajectories = {}
    for i in range(1,8):
      trajectories[i] = {}
    settings = {}
    settings['binsz']       = 0.025
    settings['plot_width']  = 400
    settings['plot_height'] = 400

    binSize_In_cm = 2.5
    binsz = binSize_In_cm / 100.0
    settings['binsz'] = binsz
    Trj = {}
    Trj1 = False
    Trj2 = False
    Trj3 = False
    Trj4 = False
    Trj5 = False
    Trj6 = False
    Trj7 = False

    Trj[1] = Trj1
    Trj[2] = Trj2
    Trj[3] = Trj3
    Trj[4] = Trj4
    Trj[5] = Trj5
    Trj[6] = Trj6
    Trj[7] = Trj7
    c = {1:'red', 2:'green', 3:'blue', 4:'orange', 5:'yellow', 6:'magenta',7:'black'}
    print('Default setting loaded.') 
    Trj_fn = {}
    for i in range(1,8):
      Trj_fn[i] = "---"
    
# Example:    https://docs.bokeh.org/en/latest/docs/gallery/stocks.html
def ppk_plot(t, x, y, z, title, x_title, y_title):
  radii = .1  
  p1 = figure( title=title )                   #
  p1.xaxis.axis_label = x_title
  p1.yaxis.axis_label = y_title
  p1.circle_cross(x,y, size=1)    # Plot the lat, lon
  
  p2 = figure()                   # 
  p2.circle_cross(                          # Plot the elevations vs time
      pd.to_datetime(ppk_data['hms_z']),
      ppk_data['elev'], 
      size=1
      )

  show( 
      gridplot([[p1,p2]], 
              plot_width=settings['plot_width'], 
              plot_height=settings['plot_height']
              ) 
      );

TOOLS = 'pan,wheel_zoom,hover,box_zoom,reset,undo, redo'
def open_new_plot():
  try:
    webgl
  except:
    NameError
    WebGL = "Enabled"

  if WebGL == 'Enabled':
    p0 = figure( plot_width = settings['plot_width'], 
               plot_height= settings['plot_height'],
               output_backend = "webgl",
                     tools=TOOLS
              )
  else:
      p0 = figure( plot_width = settings['plot_width'], 
               plot_height= settings['plot_height'],
                     tools=TOOLS
              )
  p0.toolbar.autohide                 = True
  p0.title.text_font_size             = '18pt'
  p0.yaxis.axis_label_text_font_size  = '16pt'
  p0.yaxis.major_label_text_font_size = '14pt'
  p0.xaxis.major_label_text_font_size = '14pt'
  p0.xaxis.axis_label_text_font_size  = '16pt'
  return p0

def open_time_plot():
  p0 = open_new_plot()
  p0.yaxis.axis_label                 = "Elevation (meters)"
  p0.xaxis.axis_label                 = "Time"
  p0.xaxis.major_label_orientation    = 1.
  p0.xaxis.formatter = bkm.DatetimeTickFormatter(hours=['%H:%M:%S'], 
                                                minutes=['%H-%M-%S'], 
                                                seconds=['%H_%M_%S']
                                                )
  return p0

def no_ppk_data_loaded():
  print('No ppk_data is loaded.')

# Display:
#  Min Max Nsats
def gen_header( g ):
  headers = { 1:'Number',    2:'File Name', 3:'Records', 4:'Seconds\nOffset', 5:'Start Time',
              6:'Stop Time', 7:'Duration',  8:'Generating\nSoftware', 9:'Trajectory\nType', 10:'User\nZ Bias'}
  for i in range(1,11):
    with g.output_to(0,i):
      print(headers[i])
  return g

def gen_record(g, t, n):
  r = t[n]
  with g.output_to(1,1):
    print(n)
  with g.output_to(1,2):
    print(os.path.split(r['ifn'])[1])
  with g.output_to(1,3):
    print(r['data']['lat'].count())
  with g.output_to(1,4):
    print(r['Seconds_Offset'])
  with g.output_to(1,5):
    print(r['data']['hms_z'].min())
  with g.output_to(1,6):
    print(r['data']['hms_z'].max())
  with g.output_to(1,7):
    print(r['data']['hms_z'].max() - r['data']['hms_z'].min())  
  with g.output_to(1,8):
    print(r['Generating_Software'])
  with g.output_to(1,9):
    print(r['Trajectory_Type'])
  with g.output_to(1,10):
    print(r['z_bias'])


def display_trj_stats( t, n ):
  grid = widgets.Grid(2,11, header_row=True, header_column=True)
  gen_header(grid)
  gen_record(grid, t, n)

def display_all_trj_stats():
  grid = widgets.Grid(8,11, header_row=True, header_column=True)
  gen_header(grid)
  for i in range(1,8):
    t = trajectories[i]
    if 'ifn' in t:
      gen_record(grid, trajectories, i )


##############################################################
# Begin configuring for RTKlib and processing GNSS Data      #
##############################################################
!rm -rf /content/sample_data
!cd /usr/local/bin; rm  -rf convbin rnx2rtkp pos2kml rtkrcv str2str
!cd /content/;       rm -rf RTKLIB
!cd /usr/local/src/; rm -rf RTKLIB
rv = !cd /usr/local/src; git clone https://github.com/lidar532/RTKLIB.git
rv = !cd /usr/local/src/RTKLIB/app/; make install
!cd /usr/local/bin; chmod uog+x rnx2rtkp convbin pos2kml rtkrcv str2str

# Extract the help files from each RTKlib tool.
#!convbin  -? 2> convbin.txt
#!rnx2rtkp -? 2> rnx3rtkp.txt
#!pos2kml  -? 2> pos2kml.txt
#!rtkrcv   -? 2> rtkrcv.txt 
#!str2str  -? 2> str2str.txt

# Get and install teqc
rv = !wget https://www.unavco.org/software/data-processing/teqc/development/teqc_Lx86_64s.zip
rv = !unzip teqc_Lx86_64s.zip
rv = !mv teqc /usr/local/bin
!rm -rf teqc_Lx86_64s.zip

# Install geprinex package.
!pip install georinex

# setup the options file that will be sent to the RTKlib processor.
k_settings = { 
    'pos1-soltype'      : 'combined     # (0:forward,1:backward,2:combined)',     
    'pos1-posmode'      : 'kinematic    # (0:single,1:dgps,2:kinematic,3:static,4:movingbase,5:fixed,6:ppp-kine,7:ppp-static)',
    'pos1-frequency'    : 'l1+l2        # (1:l1,2:l1+l2,3:l1+l2+l5)',        
    'pos1-elmask'       : 12,             
    'pos1-snrmask'      : 5.0,
    'pos1-dynamics'     : 'off',
    'pos1-tidecorr'     : 'off',
    'pos1-ionoopt'      : 'brdc         # (0:off,1:brdc,2:sbas,3:dual-freq,4:est-stec)',
    'pos1-tropopt'      : 'saas         # (0:off,1:saas,2:sbas,3:est-ztd,4:est-ztdgrad)',
    'pos1-sateph'       : 'precise      # (0:brdc,1:precise,2:brdc+sbas,3:brdc+ssrapc,4:brdc+ssrcom)',
    'pos1-exclsats'     : '',
    'pos1-navsys'       : '5            # (1:gps+2:sbas+4:glo+8:gal+16:qzs+32:comp)',
    'pos2-armode'       : 'continuous   # (0:off,1:continous,2:instantaneous,3:fix-and-hold)',
    'pos2-gloarmode'    : 'on',
    'pos2-arthres'      : 3,
    'pos2-arlockcnt'    : 5,
    'pos2-arelmask'     : 0,
    'pos2-aroutcnt'     :5,
    'pos2-arminfix'     :10,
    'pos2-slipthres'    :0.05,
    'pos2-maxage'       :30,
    'pos2-rejionno'     :30,
    'pos2-niter'        :1,
    'pos2-baselen'      :0,
    'pos2-basesig'      :0,
    'out-solformat'     :'llh           # (0:llh,1:xyz,2:enu,3:nmea)',
    'out-outhead'       :'on',
    'out-outopt'        :'on',
    'out-timesys'       :'utc           # (0:gpst,1:utc,2:jst)',
    'out-timeform'      :'hms           # (0:tow,1:hms)'        ,
    'out-timendec'      :6,
    'out-degform'       :'deg'        ,
    'out-fieldsep'      : '',
    'out-height'        :'ellipsoidal' ,
    'out-geoid'         :'internal'   ,
    'out-solstatic'      :'all          # (0:all,1:single)'        ,
    'out-nmeaintv1'      :0          ,
    'out-nmeaintv2'      :0          ,
    'out-outstat'        :'off'        ,
    'stats-errratio'     :100,
    'stats-errphase'     :0.003      ,
    'stats-errphaseel'   :0.003      ,
    'stats-errphasebl'   :0          ,
    'stats-errdoppler'   :10         ,
    'stats-stdbias'      :30         ,
    'stats-stdiono'      :0.03       ,
    'stats-stdtrop'      :0.3        ,
    'stats-prnaccelh'    :1          ,
    'stats-prnaccelv'    :0.1        ,
    'stats-prnbias'      :0.0001     ,
    'stats-prniono'      :0.001      ,
    'stats-prntrop'      :0.0001     ,
    'stats-clkstab'      :5e-12      ,
    'ant1-postype'       :'llh       # (0:llh,1:xyz,2:single,3:posfile,4:rinexhead,5:rtcm)'      ,
    'ant1-pos1'          :0          ,
    'ant1-pos2'          :0          ,
    'ant1-pos3'          :0          ,
    'ant1-anttype'       :'*',
    'ant1-antdele'       :0          ,
    'ant1-antdeln'       :0          ,
    'ant1-antdelu'       :0          ,
    'ant2-postype'       :'rinexhead          # (0:llh,1:xyz,2:single,3:posfile,4:rinexhead,5:rtcm)',
    'ant2-pos1'          :'35.1320570679997   # lat',
    'ant2-pos2'          :'139.624306577      # Lon',
    'ant2-pos3'          :'73.907699999947    # Elevation' ,
    'ant2-anttype'       :'*',
    'ant2-antdele'       :0          ,
    'ant2-antdeln'       :0          ,
    'ant2-antdelu'       :0          ,
    'misc-timeinterp'    :'on'         ,
    'misc-sbasatsel'     :0          ,
    'file-satantfile'    :'/usr/local/src/RTKLIB/data/igs05.atx',
    'file-rcvantfile'    :'/usr/local/src/RTKLIB/data/igs05.atx',
    'file-staposfile'    :'/usr/local/src/RTKLIB/data//stations.pos',
    'file-geoidfile'     : '',
    'file-dcbfile'       :'/usr/local/src/RTKLIB/data/P1C1_ALL.DCB',
    'file-tempdir'       : '/tmp/',
    'file-geexefile'     : '',
    'file-solstatfile'   : '',
    'file-tracefile'     : ''
    }

rtk_lib_loaded = True
########################################################
#   END stuff for RTKlib and processing GNSS Data      #
########################################################

########################################################
#   Begin configuring the plotting tools               #
########################################################
output_notebook()
Google_Maps_Key = "Replace this with your Google Maps API Key"
init_analysis()
print('The Toolbox is ready for use.')
########################################################
#   END configuring the plotting tools                 #
########################################################

!apt-get install tree

print( "******************************************" )
print( "* Ready to process with RTKlib & teqc    *" )
print( "* Plotting tools are ready to use also.  *" )
print( "******************************************" )


In [0]:
#@title A.0.2 **(Optional)** Setup A new Project Directory Tree
My_Project = '2019-0326-DFC-Solo2' #@param {type:"string"}

rv = !rm -rf /content/{My_Project}
rv = !mkdir /content/{My_Project}
rv = !cd {My_Project};\
 mkdir 01_gps \
  01_gps/aircraft \
  01_gps/aircraft/exif \
  01_gps/aircraft/raw \
  01_gps/bases \
  01_gps/trajectories \
  01_gps/pix_pos \
  02_Ground_Control \
  03_Photos \
  03_Photos/raw \
  03_Photos/jpg \
  04_Agisoft \
  05_Products \
  05_Products/Reports \
  05_Products/laz \
  05_Products/dsm \
  05_Products/ortho \
  06_Metadata 
!tree {My_Project}
print(My_Project," is setup.\nOperation Complete.")


In [0]:
#@title A.0.3: Convert a CWW-PPK raw GPS data file to RINEX for processing. {form-width: "25%"}

#@markdown The first thing that must be done is converting your raw GNSS receiver data to RINEX.
#@markdown Use this cell to convert CWW-PPK raw data files to RINEX.  You need this for
#@markdown the UAS PPK data.  If you are using a second CWW-PPK in "Base Station" mode, you can
#@markdown it for that also.  If you are using another survey grade receiver, you will probably
#@markdown need to use tools specific to that receiver to convert it to RINEX.  RINEX is the required
#@markdown data format for use with this software package.
Raw_File_Name = "/content/2019-0326-DFC-Solo2/01_gps/aircraft/raw/GP213208.RAW" #@param {type:"string"}
Data_Source   = "Solo" #@param ["Solo", "M600", "N7251F", "CCX", "TFR"] {allow-input: true}
Marker_Name   = "Solo" #@param {type:"string"}
User_Comment  = "Denver Fed Center. 3DR Solo Ricoh GR-II Joe Adams." #@param {type:"string"}

try:
  rtk_lib_loaded
  import re
  import os as os
  import georinex as gr
except NameError:
  print('Run the above "**A0.0 **" cell above to configure tools.')
else:
  if os.path.exists(Raw_File_Name):
    # Build up command string for the RTKlib convbin program.
    c1='-hc \''+User_Comment+'\' '
    c2='-hc \''+Data_Source+'\' '
    hm='-hm \''+Marker_Name+'\' '
    fn_root = str.split(Raw_File_Name,"/")[-1]
    fn_root = str.split(fn_root,'.')[0]
    cmd = '/usr/local/bin/convbin -os -od -f 2 -ti 1 -r nov '+c1+hm+Raw_File_Name
    print(cmd)
    print('Converting raw to RINEX')
    os.system( cmd )
    #!head -20 /content/GP212112.obs


    # Extract the base file name after all '/' and before any '.'
    orig_root_fn = os.path.basename(Raw_File_Name)
    orig_root_fn = os.path.splitext(orig_root_fn)[0]
    orig_path  = os.path.dirname(Raw_File_Name)

    # build up path name for the RINEX .obs file that should now exist
    fobs = orig_path+'/'+orig_root_fn+'.obs'

    # extract the RINEXT Header information to get the data & time and other fields
    print('Reading the RINEX obs, extracting data & time.')
    rnx=gr.rinexheader(fobs)
    rnx['t0'].date(), rnx['t0'].time()
    ofn = format(rnx['t0'], '%Y-%m%d-%H%M%S-')+Data_Source+'-'+fn_root

    # Rename all of the generate RINEX files using the computed root name.
    d = os.listdir(orig_path)
    print('Renaming all of the RINEX files')
    for i in d:
      if re.match( os.path.splitext(i)[0], orig_root_fn  ):
        ext = os.path.splitext(i)[1]
        new_fn = orig_path+'/'+ofn+ext
        new_fn = re.sub(r"[\n\t\s]*", "", new_fn)
        old_fn = orig_path+'/'+i
        print(old_fn, new_fn)
        os.rename(old_fn, new_fn)
    print('Operation completed.')
  else:
    print(Raw_File_Name+' not found.')

In [0]:
#@title A.0.4 **(Required)** Load Base Station RINEX Files and settings to use. {form-width: "35%"}

# convert the lat/lon strings found in OPUS reports to usable float values
def dms2dd( dms ):
  rv = -999
  strt = type('x')
  if type(dms) == strt:
    dms = dms.split()  
  if type(dms) == float or type(dms)  == int:
    rv =  float(dms)
  elif len(dms) == 1:
    rv = float(dms[0])
  elif len(dms) == 5:
    dms[0] = dms[0].upper()
    rv = int(dms[2]) + int(dms[3])/60.0 + float(dms[4])/3600.0
    if ( dms[0] == 'S' or dms[0] == 'W'):
      rv = -rv
    return rv
  elif len(dms) == 4:
      dms[0] = dms[0].upper()
      rv = int(dms[1]) + int(dms[2])/60.0 + float(dms[3])/3600.0
      if ( dms[0] == 'S' or dms[0] == 'W' ):
        rv = -rv
  return rv


Ephemeris_Type = "Precise"         #@param ['Precise', 'Rapid', 'Ultra Rapid', 'Broadcast' ]
Satellite_Systems = 'GPS+GLONASS'  #@param ['GPS', 'GPS_SBAS', 'GPS+GLONASS' ]
Base_RINEX      = "/content/2019-0326-DFC-Solo2/01_gps/bases/ctmc0850.19o"           #@param {type:"string"}
Base_RINEX_Nav  = "/content/2019-0326-DFC-Solo2/01_gps/bases/ctmc0850.19n"           #@param {type:"string"}
Base_RINEX_Gnav = "/content/2019-0326-DFC-Solo2/01_gps/bases/ctmc0850.19g"           #@param {type:"string"}
Base_GPS_SP3    = "/content/2019-0326-DFC-Solo2/01_gps/bases/igs20462.sp3"           #@param {type:"string"}          
Base_GLO_SP3    = "/content/2019-0326-DFC-Solo2/01_gps/bases/igl20462.sp3"           #@param {type:"string"}
Base_data_ready_for_use = True;

#@markdown ---
#@markdown Select 'RINEX Header' if you are using 
#@markdown [CORS](https://www.ngs.noaa.gov/CORS_Map/) data as your base station.
#@markdown Select 'Lat Lon Elevation' or 'X Y Z' if you are using your own local base station. 
#@markdown 'X Y Z' coordinates are 'Earth Centered Earth Fixed' (ECEF).
Base_Coordinates_Source = 'RINEX Header' #@param ['RINEX Header', 'Lat Lon Elevation', 'X Y Z', 'OPUS Report' ]
Base_Longitude_X = ''   #@param {type:"string"}
Base_Latitude_Y  = '' #@param {type:"string"}
Base_Elevation_Z = ''  #@param {type:"string"} 

#@markdown Specify where to get the precise coordinates from.  
#@markdown Select RINEX if you are using [CORS](https://www.ngs.noaa.gov/CORS_Map/)
#@markdown data for your base station.  If you are using your own local base station, enter
#@markdown the antenna identifier for your antenna.
Base_Antenna = "RINEX Header" #@param ["RINEX Header", "CNTAT330", "Use Phase Center"] {allow-input: true}
OPUS_Report_File = "" #@param {type:"string"}
Base_Position_Type  = 'Not Set'
# base_files_exist will ultimately remain true only if all of the required files
# exist.  It does not check for validity (yet).
base_files_exist = True;
if  os.path.exists(Base_RINEX) == False:
  print('Base_RINEX file not found: ', Base_RINEX)
  Base_data_ready_for_use = False
  base_files_exiss = False

if  os.path.exists(Base_RINEX_Nav) == False:
  print('Base_RINEX_Nav file not found: ', Base_RINEX_Nav)
  base_files_exist = False
  Base_data_ready_for_use = False

if  (Satellite_Systems=='GPS+GLONASS'):
  if os.path.exists(Base_RINEX_Gnav) == False:
    Base_data_ready_for_use = False
    print('Gnav file not found: ', Base_RINEX_Gnav)
    base_files_exist = False
    Base_data_ready_for_use = False
  if os.path.exists(Base_GLO_SP3) == False:
    print('Gnav SP3 file not found: ', Base_GLO_SP3)
    base_files_exist = False
    Base_data_ready_for_use = False

if Base_Coordinates_Source == 'RINEX Header':
  Base_Position_Type = 'rinexhead'
elif Base_Coordinates_Source == 'X Y Z':
  Base_Position_Type = 'xyz'
elif Base_Coordinates_Source == 'Lat Lon Elevation':
  Base_Position_Type = 'llh'



if Base_Coordinates_Source=='OPUS Report':    # if an OPUS file is found, then get coords from it
  if os.path.exists(OPUS_Report_File):
    opus_lat = get_file_var( OPUS_Report_File,'LAT:'  ).split()[0:4]
    opus_lon = get_file_var( OPUS_Report_File,'W LON:'  ).split()[0:5]
    opus_elev = get_file_var(OPUS_Report_File,'EL HGT:'  ).split()[2]
    opus_elev = float(opus_elev.split('(')[0])
    Base_Position_Type  = 'llh'
    Base_Latitude_Y  = opus_lat
    Base_Longitude_X = opus_lon
    Base_Elevation_Z = opus_elev
    k_settings['ant2-pos1']    = Base_Latitude_Y
    k_settings['ant2-pos2']    = Base_Longitude_X
    k_settings['ant2-pos3']    = Base_Elevation_Z
    k_settings['ant2-postype'] = Base_Position_Type
    print('Using the NAD83 Ellipsodial Coordinates from your OPUS Report File for your base station.')
  else:
    print('No OPUS Report file found.')
    Base_data_ready_for_use = False

if base_files_exist:
  strx = 'not set'
  if Base_Coordinates_Source != 'RINEX Header':
    Base_Latitude_Y = dms2dd(Base_Latitude_Y)
    Base_Longitude_X = dms2dd( Base_Longitude_X)
    Base_Elevation_Z = dms2dd( Base_Elevation_Z)
    Base_Position_Type = 'llh'
    if Base_Latitude_Y  < -90.0 or Base_Latitude_Y > 90.0:
      print(Base_Latitude_Y, 'Base_Latitude_X is invalid.')
    elif Base_Longitude_X  < -360.0 or Base_Longitude_X > 360.0:
      print(Base_Longitude_X, 'Base_Longitude_X is invalid.')
    elif Base_Elevation_Z < -900:
      print(Base_Elevation_Z, 'Base_Elevation_Z is invalid.')
    else:
      strx = "Base Station Coordinates: {:s}  Latitude: {:6.9F}  Longititude: {:6.9F} Elevation: {:6.3F} (m)".\
      format( Base_Position_Type, 
              Base_Latitude_Y, 
              Base_Longitude_X,
              Base_Elevation_Z )
      if Base_data_ready_for_use == True:
        k_settings['ant2-pos1']    = Base_Latitude_Y
        k_settings['ant2-pos2']    = Base_Longitude_X
        k_settings['ant2-pos3']    = Base_Elevation_Z
        k_settings['ant2-postype'] = Base_Position_Type
        print(strx )
  else:
    print('Using coordinates in the RINEX Header.')
    k_settings['ant2-postype'] = 'rinexhead'
  if Base_data_ready_for_use == True:
    print('Operation Completed ok.  Ready to proceed.')
else:
  print('Please correct and retry.')
  Base_data_ready_for_use = False 

In [0]:
#@title A.0.5 Display A File.
##show_file_head = "Base_RINEX" #@param ['Base_RINEX', 'Base_RINEX_Nav', 'Base_GPS_SP3', 'Base_RINEX_Gnav', 'Base_GLO_SP3' ]
show_file_head = "/content/2019-0326-DFC-Solo/01_gps/bases/ctmc0850.19o" #@param {type:"string"}
Lines_to_display = 40 #@param ["20", "30", "40", "50", "75", "100"] {type:"raw", allow-input: true}


!head -{Lines_to_display} {show_file_head}


In [0]:
#@title A.0.6 QC a RINEX Observation (*.Obs or *.19o) file.
#@markdown See [Page 34 of the UNAVCO Teqc Tutorial](https://www.unavco.org/software/data-processing/teqc/doc/UNAVCO_Teqc_Tutorial.pdf) for help interpreting the output.
OBS_File_QC = "/content/2019-0326-DFC-Solo/01_gps/aircraft/raw/2019-0326-213228-Solo-GP213208.obs" #@param {type:"string"}

!teqc +qc {OBS_File_QC} 2>/content/errors.txt

In [0]:
#@title A0.7 Display the Kinematic Processing Settings.
for k in k_settings:
  print(f'{k:<16}  :{k_settings[k]:<60}')

## A.1 When you setup a local base station

In [0]:
#@title A1.1 Prepare Base RINEX to send to OPUS or to use with RTKlib (static). { form-width: "35%" }
#@markdown If you setup a local base station using a 
#@markdown second CWW-PPK or other GNSS receiver you can
#@markdown use this cell to generate the necessary RINEX
#@markdown data files to send to 
#@markdown [OPUS](https://www.ngs.noaa.gov/OPUS/).
#@markdown [OPUS](https://www.ngs.noaa.gov/OPUS/) gives you
#@markdown the highest accuracy location
#@markdown of your base station.
#@markdown This cell can process raw CWW-PPK data or RINEX
#@markdown data from another GNSS receiver.
#@markdown It will  re-sample your base station data
#@markdown to a specified interval. A 30 second interval is 
#@markdown recommended.
#@markdown It is important to use the correct antenna type ID when
#@markdown when you process this.  Not doing so can cause
#@markdown the resulting positon to be in error by 10cm or more
#@markdown depending on your base station antenna make/model.
#@markdown Much more information on OPUS canbe found in this document: [OPUS_2012_MALSCE.pdf](https://outside.vermont.gov/agency/vtrans/external/docs/geodetic/OPUS_2012_MALSCE.pdf)



Base_Marker_Name = "CWW_PPK_BASE" #@param ["CWW_PPK_BASE", ""] {allow-input: true}
OPUS_Sample_Interval_Seconds =  30 #@param {type:"integer"}
Base_Antenna_Type = "TRM55971.00" #@param ["CNTAT330", ""] {allow-input: true}
Base_Receiver_Data_Format = "RINEX" #@param ["Raw-CWW-PPK", "RINEX"] {allow-input: true}
Base_Raw_File = "/content/2019-0326-DFC-Solo2/01_gps/bases/ctmc0850.19o" #@param {type:"string"}
Base_User_Comment = "CTMC TRM55971.00 " #@param {type:"string"}

Base_Rx_Types = {
    'NovAtel OEMV/4/6,OEMStar'    : 'nov',
    'NovAtel OEM3'                : 'oem3',
    'ublox LEA-4T/5T/6T/7T/M8T'   : 'ubx',
    'Swift Navigation SBP'        : 'sbp',
    'Hemisphere Eclipse/Crescent' : 'hemis',
    'SkyTraq S1315F'              : 'stq',
    'Javad'                       : 'javad',
    'Trimble RT17'                : 'rt17',
    'Septentrio SBF'              : 'sbf',
    'BINEX'                       : 'binex',
    'TERSUS'                      : 'tersus',
    'RINEX'                       : 'rinex'
    }

Rx_fmt = { 
          'Raw-CWW-PPK'   : 'nov',
          'RINEX'         : 'rinex'
          }


opus_dir = os.path.dirname( Base_Raw_File )+'/opus'
print('Generating RINEX data for OPUS.')
!/usr/local/bin/convbin \
  -r {Rx_fmt[Base_Receiver_Data_Format]} \
  -hm {Base_Marker_Name} \
  -ti {OPUS_Sample_Interval_Seconds} \
  -tt 0.02 \
  -ha {Base_Antenna_Type} \
  -c {Base_Marker_Name} \
  -d {os.path.dirname( Base_Raw_File )+'/'}opus \
  {Base_Raw_File} 2> errors.txt
!grep -h 'TIME OF' {opus_dir+'/*'}
!grep -h 'ANT #'   {opus_dir+'/*'}
print(opus_dir)
! ls -la {opus_dir}
print('Operation completed.  Download the generated observation data and send to OPUS.')


## A.2. GNSS PPK for / UAS / Manned Aircraft / GPS Trajectory Processing

In [0]:
#@title **A Step 3:** Generate an RTKlib GNSS Trajectory. { form-width: "35%" }



Position_Mode  = "kinematic"       #@param ['kinematic', 'single', 'dgps', 'static', 'fixed', 'ppp-static']
Solution_Type  = "combined"        #@param ['combined', 'forward', 'backward' ]
Elevaton_Mask =  20#@param ["5", "10", "12", "15", "18", "20", "22", "25", "27", "30", "33", "35", "40"] {type:"raw", allow-input: true}
Aircraft_RINEX = "/content/2019-0326-DFC-Solo2/01_gps/aircraft/raw/2019-0326-213228-Solo-GP213208.obs"         #@param {type:"string"}
Trajectory_Folder = "/content/2019-0326-DFC-Solo2/01_gps/trajectories" #@param {type:"string"}
Generate_Output_Trajectory = "Yes" #@param ["Yes", "No"]

eph_types = { 'Precise':    'precise',
              'Rapid' :     'precise',
              'Ultra Rapid':'precise',
              'Broadcast' : 'brdc'
              }
svs = { 'GPS' : 1, 
        'GPS_SBAS' : 3,
        'GPS+GLONASS' : 5 }


# Copy user configured settings to the configuration data structure.
k_settings['pos1-sateph' ] = eph_types[Ephemeris_Type]
k_settings['pos1-posmode'] = Position_Mode
k_settings['pos1-soltype'] = Solution_Type
k_settings['pos1-elmask' ] = Elevaton_Mask
k_settings['pos1-navsys' ] = svs[ Satellite_Systems ]

if Base_Coordinates_Source == 'RINEX Header':
  k_settings['ant2-postype' ] = 'rinexhead'  
elif Base_Coordinates_Source == 'Lat Lon Elevation': 
  k_settings['ant2-postype' ] = 'llh'
  k_settings['ant2-pos1'    ] = Base_Latitude_Y
  k_settings['ant2-pos2'    ] = Base_Longitude_X
  k_settings['ant2-pos3'    ] = Base_Elevation_Z
elif Base_Coordinates_Source == 'X Y Z':
  k_settings['ant2-postype' ] = 'xyz'
  k_settings['ant2-pos1'    ] = Base_Latitude_Y
  k_settings['ant2-pos2'    ] = Base_Longitude_X
  k_settings['ant2-pos3'    ] = Base_Elevation_Z

try:
  rtk_lib_loaded
  import re
  import os as os
  import georinex as gr
  import pathlib
except NameError:
  print('****************************************************************************************')
  print('*    Run the above "A.0.0" cell above to configure tools before using this cell.   *')
  print('****************************************************************************************') 
else:
  if os.path.exists(Aircraft_RINEX):
    # Build up the output Trajectory file name
    bn = get_file_var(Base_RINEX, 'MARKER NAME').split()[0]
    otfn_mode  =   {'kinematic' : 'K', 'single'   : 'C',        'dgps' : 'D',    'static' : 'S', 'fixed' : 'F', 'ppp-static' : 'PPPS'}
    otfn_etype =   {'Precise'   : 'P', 'Rapid'    : 'R', 'Ultra Rapid' : 'U', 'Broadcast' : 'B' }
    otfn_navsys=   {'GPS'       : 'N', 'GPS_SBAS' : 'W', 'GPS+GLONASS' : 'GN' }
    otfn_direction={'combined'  : 'cmb', 'forward': 'fwd', 'backward'  : 'rev'}

    otfn_root = str.split(
        os.path.basename(Aircraft_RINEX),
         '.')[0] + '-'+\
         bn+'-'+ \
         otfn_etype[Ephemeris_Type] +  \
         otfn_mode[Position_Mode] + \
         str(k_settings['pos1-elmask']) + \
         otfn_navsys[Satellite_Systems] + '-' + \
         otfn_direction[k_settings['pos1-soltype']]

    # Write the custom configured options out to a configuration file.
    otfn_conf = Trajectory_Folder+'/'+otfn_root+'-conf.txt'   
    otf_conf = open(otfn_conf, 'w+' )
    for i in k_settings:
      print( i, '\t\t=', k_settings[i], file=otf_conf, sep='' )
    otf_conf.close()
    
    otfn     = Trajectory_Folder+'/'+otfn_root+'-RTKtraj-pos.txt'
    otfn_kml = Trajectory_Folder+'/'+otfn_root+'-RTKtraj-pos-flat.kml'
    otfn_kmz = Trajectory_Folder+'/'+otfn_root+'-RTKtraj-pos-flat.kmz'
    otfna_kml = Trajectory_Folder+'/'+otfn_root+'-RTKtraj-pos-elev.kml'
    otfna_kmz = Trajectory_Folder+'/'+otfn_root+'-RTKtraj-pos-elev.kmz'    
    # Build up command string for the RTKlib post processor program.
    # Use the root file name of the Aircraft file.
    cmd = '/usr/local/bin/rnx2rtkp  -t \\\n\
        -k '+otfn_conf+' \\\n\
        -o ' + otfn +          '\\\n\t' \
           + Aircraft_RINEX + " \\\n\t" \
           + Base_RINEX +     " \\\n\t" \
           + Base_RINEX_Nav + " \\\n\t" \
           + Base_RINEX_Gnav + " \\\n\t" \
           + Base_GPS_SP3 +   " \\\n\t" \
           + Base_GLO_SP3 +   " \\\n\t" \
           + " 2>errors.txt >stdout.txt"
    print(cmd)
    if Generate_Output_Trajectory == 'Yes':
      print('**************************************************')
      print('* Processing.  This may take several minutes     *')
      print('**************************************************') 
      os_rv = os.system( cmd )

      # Generate a kml from trajectory file.
      rv = !pos2kml  -tu -q 0 -o     {otfn_kml}  {otfn}
      rv = !pos2kml  -tu -ag -q 0 -o {otfna_kml} {otfn}
      #convert the kml to a kmz
      !zip -q {otfn_kmz} {otfn_kml};    rm {otfn_kml}
      !zip -q {otfna_kmz} {otfna_kml};  rm {otfna_kml}      
      if rv: 
        print(rv)
      print('Trajectory generated.  Operation completed.')
      #!cat errors.txt
    else:
      print('No output file generate.')
      print('Command: ', cmd)
  else:
    print(Aircraft_RINEX+' not found.')


**B:** Plot & Analysis Tools 
===

## B.0: Load GNSS Trajectories Files for Analysis


### B.0.1: RTKLIB: Load Trajectory Files.

In [0]:
#@title B.0.1: RTKLib Load File.
traj_target = 2 #@param [1,2,3,4,5,6,7] {type:"raw"}
RTKLIB_Seconds_Offset = 0 #@param {type:"integer"}
User_Z_Bias = 0.0 #@param {type:"raw"}

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  """
  Loads a .pos "Position file" generated by RTKLIB into the pandas dataframe ppk_trj.  The .pos file
  is expected to contain UTC time and not GPS time.
  """
  PPK_File_Name = "paste ppk file name here."
  PPK_File_Name = "/content/2019-0326-DFC-Solo2/01_gps/trajectories/2019-0326-213228-Solo-GP213208-CTMC-PK12GN-cmb-RTKtraj-pos.txt" #@param {type:"string"}
  Trajectory_Type = "PPK" #@param ["PPK", "RTK", "PPP", "Code", "Code+SBAS"] {allow-input: true}


  f = pathlib.Path(PPK_File_Name)
  if f.exists() == False:
    print(f," does not exist.")
  else:
    Trj_fn[traj_target]=PPK_File_Name
    ppk_trj = pd.read_csv( PPK_File_Name, 
                        names=['date', 'zhms', 
                                'GPS_lat', 'GPS_lon', 'GPS_nad83h','Q', 'ns',
                                'sdn', 'sde', 'sdu', 'sdne', 'sdeu', 'sdun', 'age', 'ratio'],
                        delim_whitespace=True,
                        comment='%',
                        skiprows = 1 )
    # ppk_trj
    ppk_trj['Ztime'] = pd.to_datetime( ppk_trj['date']+' '+ppk_trj['zhms']) \
                      + + pd.to_timedelta(RTKLIB_Seconds_Offset, unit='seconds')
    ppk_trj['Ztime_idx'] = ppk_trj['Ztime']
    ppk_trj = ppk_trj.set_index('Ztime_idx')

    data = pd.DataFrame()
    data['hms_z'] = ppk_trj['Ztime']
    data['lat']   = ppk_trj['GPS_lat']
    data['lon']   = ppk_trj['GPS_lon']
    data['elev']  = ppk_trj['GPS_nad83h'] + User_Z_Bias
    data['ns']    = ppk_trj['ns']
    data['sdu95'] = ppk_trj['sdu']*1.95
    data['q']     = ppk_trj['Q']
    data['gdop']  = 0
    trajectories[traj_target]['ifn'] = PPK_File_Name
    trajectories[traj_target]['data'] = data
    trajectories[traj_target]['z_bias']  = User_Z_Bias
    trajectories[traj_target]['Generating_Software'] = "RTKlib"
    trajectories[traj_target]['Trajectory_Type'] = Trajectory_Type
    trajectories[traj_target]['Seconds_Offset']    = RTKLIB_Seconds_Offset
    #display_trj_stats( trajectories, traj_target )
    display_all_trj_stats()

   # %load_ext google.colab.data_table
   # display( data )
   # %unload_ext google.colab.data_table




### B.0.2: PPP-CA: Load Trajectory Files.


In [0]:
#@title B.0.2: Load PPP-Ca Trajectory {display-mode: "form"}
#@markdown Use the [free online PPP processing software from the Natural Resources Canada  ](https://webapp.geod.nrcan.gc.ca//geod/account-compte/login.php?locale=en "Free PPP") server to generate a precision GNSS trajectory from your RINEX UAS data.

def load_ppkca_trj( ppk, ppk_user_settings ):
  f = pathlib.Path(ppk_user_settings['Trajectory_file_Name'] )
  if f.exists() == False:
    print(f," does not exist.")
    ppk['run'] = False
  else:
    ppp_pos = pd.read_csv(ppk_user_settings['Trajectory_file_Name'], skiprows=5, sep='\s+' )
    Trj_fn[traj_target] = ppk_user_settings['Trajectory_file_Name']
    ppk_data = {}
    ppk_data['hms_z']         = pd.to_datetime( ppp_pos['YEAR-MM-DD'] +" "+ppp_pos['HR:MN:SS.SS'] ) \
                                + pd.to_timedelta(PPP_Seconds_Offset, unit='seconds')
    ppk_data['lat']         =     ppp_pos['LATDD']  + ppp_pos['LATMN']/60.0 + ppp_pos['LATSS']/3600.0
    ppk_data['lon']         = -( abs(ppp_pos['LONDD']) + ppp_pos['LONMN']/60.0 + ppp_pos['LONSS']/3600.0 )
    ppk_data['elev']        = ppp_pos['HGT(m)'] + User_Z_Bias
    ppk_data['sdu95']       = ppp_pos['SDHGT(95%)']
    ppk_data['gdop']        = ppp_pos['GDOP']
    ppk_data['ns']          = ppp_pos['NSV']
    ppk_data['q']           = 6;
    return ppk_data

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  traj_target = 3 #@param [1,2,3,4,5,6,7] {type:"raw"}
  User_Z_Bias = 0.0 #@param {type:"raw"}
  PPP_Seconds_Offset = -16 #@param {type:"integer"}
  ppp_pos_ifn = "/content/2018-0326-gps-TFR-ppp-flight-1.pos" #@param {type:"string"} 
  Trajectory_Type = "PPP" #@param ["PPP"] {allow-input: true}
  ppk_user_settings = {}
  ppk = {}
  ppk_user_settings['Trajectory_file_Name'] = ppp_pos_ifn
  ppk['run'] = True
  
  #============================================================================
  # Canadian PPP *.pos file reader
  #============================================================================
  init_analysis()

  trajectories[traj_target]['data'] = load_ppkca_trj( ppk, ppk_user_settings )
  trajectories[traj_target]['Generating_Software'] = "Can PPP"
  trajectories[traj_target]['Seconds_Offset']      = PPP_Seconds_Offset
  trajectories[traj_target]['Trajectory_Type']     = Trajectory_Type
  trajectories[traj_target]['ifn']     = ppp_pos_ifn
  trajectories[traj_target]['z_bias']  = User_Z_Bias
##  trajectories[traj_target]['data']    = ppk_data
 
display_all_trj_stats()


    


### B.0.3: GrafNav: Load Trajectory Files.

In [0]:
#@title B.0.3: GrafNav:  Load File.

def load_grafnav_trj( fn ):
  data = pd.DataFrame()
  ppk_trj = pd.read_csv(fn,
                    header=None, 
                    skiprows=40,
                    index_col=False, 
                    infer_datetime_format=True,
                    parse_dates=[['date','utc']],
                    delim_whitespace=True,
                    names=['UTMeasting', 'UTMnorthing', 'navd88', 'GPS_lat', 'GPS_lon', 'GPS_nad83h', 'Q', 'sdu', 'sdne', 'date', 'utc' ],
                    skipinitialspace=True )
  ppk_trj['Ztime'] = ppk_trj['date_utc']
  ppk_trj['Ztime_idx'] = ppk_trj['Ztime']
  ppk_trj = ppk_trj.set_index('Ztime_idx')
  data['hms_z'] = ppk_trj['Ztime']
  data['lat']   = ppk_trj['GPS_lat']
  data['lon']   = ppk_trj['GPS_lon']
  data['elev'] = ppk_trj['GPS_nad83h'] + User_Z_Bias
  data['q']    = ppk_trj['Q']
  data['sdu95'] = ppk_trj['sdu']*1.95
  data['z_bias']    = User_Z_Bias

  trajectories[traj_target]['Generating_Software'] = "GrafNav"
  trajectories[traj_target]['Trajectory_Type'] = Trajectory_Type
  trajectories[traj_target]['Seconds_Offset']    = GrafNav_Seconds_Offset
  trajectories[traj_target]['z_bias']  = User_Z_Bias
  trajectories[traj_target]['ifn']  = fn
  trajectories[traj_target]['data'] = data
#    print( trajectories[traj_target]['ifn'],
#           trajectories[traj_target]['data']['lat'].count()
#          )
  return

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  traj_target = 2 #@param [1,2,3,4,5,6,7] {type:"raw"}
  User_Z_Bias = 0.0 #@param {type:"raw"}
  GrafNav_Seconds_Offset = 0 #@param {type:"integer"}
  GrafNav_File_Name = "/content/cww-2019-0912-p-GP205057-epochs-ppp.txt" #@param {type:"string"}
  Trajectory_Type = "PPP" #@param ["PPK Single Base", "PPK Multi Base", "PPP", "RTK", "CA Code", "CA Code + WAAS"] {allow-input: true}
 

  f = pathlib.Path(GrafNav_File_Name)
  if f.exists() == False:
    print(f," does not exist.")
  else:
    # Trj_fn[traj_target] = GrafNav_File_Name
    load_grafnav_trj(GrafNav_File_Name)
    #display_trj_stats( trajectories, traj_target )
    display_all_trj_stats()


## **B.1:** Trajectory Tools

In [0]:
#@title B.1.0 Display loaded Trajectories
try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  display_all_trj_stats()

In [0]:
#@title B.1.1 Remove Selected Trajectory from Memory
Trajector_to_Remove = 1 #@param ["1", "2", "3", "4", "5", "6", "7"] {type:"raw"}
try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  print(Trajector_to_Remove)
  trajectories[Trajector_to_Remove] = {}
  print("Trajectory # ", Trajector_to_Remove, " Removed from Memory.")
  display_all_trj_stats()

# **C:** Plots and Graphs

## C.0 (Required before using C Cells) Plot options & Settings

In [0]:
#@title C.0.0 (Required )'Settings and Options' { run: "auto" }

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  try:
      settings
  except NameError:
      settings = {}
  #@markdown ---
  Google_Maps_Key = "Replace this with your Google Maps API Key" #@param {type:"string"}
  WebGL =  "Enabled" #@param ["Enabled", "Disabled"]
  #@markdown ---
  #@markdown #Trajectories to Plot 
  Trj1 = True #@param {type:"boolean"}
  Trj2 = True #@param {type:"boolean"}
  Trj3 = True #@param {type:"boolean"} 
  Trj4 = False #@param {type:"boolean"}
  Trj5 = False #@param {type:"boolean"}
  Trj6 = False #@param {type:"boolean"}
  Trj7 = False #@param {type:"boolean"}
  #@markdown ---
  #@markdown  **Plot Options**
  plot_dots = True #@param {type:"boolean"}
  plot_lines = True #@param {type:"boolean"}

  #@markdown ---
  #@markdown **Graphic Window Size (Pixels)**
  Width = 900 #@param {type:"slider", min:200, max:1000, step:10}
  Height = 500 #@param {type:"slider", min:200, max:1000, step:10}
  settings['plot_width'] =  Width
  settings['plot_height'] = Height
  Line_Width = 2 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
  #@markdown ---
  #@markdown **Histogram Bin size** 

  binsz = binSize_In_cm / 100.0
  binSize_In_cm = 2.5 #@param {type:"slider", min:1, max:100, step:0.5}
  Trj[1] = Trj1
  Trj[2] = Trj2
  Trj[3] = Trj3
  Trj[4] = Trj4
  Trj[5] = Trj5
  Trj[6] = Trj6
  Trj[7] = Trj7

  print('Operation Completed')
  

## C.1 Generate  Plots / Graphs

In [0]:
#@title Plot Elevations vs Time
import bokeh.plotting as bkp
import bokeh.models as bkm

Elevation_Dif_Plot_Title = "Elevations vs Time " #@param {type:"string"}

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  display_all_trj_stats()
  p0 = open_time_plot()
  p0.title.text = Elevation_Dif_Plot_Title
  for T in Trj:
    if T in trajectories and Trj[T] == True and 'data' in trajectories[T]:
      #print(T, c[T], Trj_fn[T])
      dt =  trajectories[T]['data']
      if plot_dots:
        p0.circle_cross(                          # Plot the elevations vs time
        pd.to_datetime(dt['hms_z']),
        dt['elev'], 
        size=4,
        color=c[T],
        )
      if plot_lines:
        p0.line(                          # Plot the elevations vs time
        pd.to_datetime(dt['hms_z']),
        dt['elev'],
        legend=str(T),
        line_width=Line_Width, 
        color=c[T]
        )

  show(p0)

In [0]:
#@title Plot Elevation Histograms
# Based on: https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram.html

def ppk_zhist(p1, x, z, c, lbl):
  p1.xaxis.axis_label = "Elevation Meters"
  if plot_lines:
    p1.line(x,z, 
            color=c, 
            line_width=Line_Width,
            legend=lbl
            )
  if plot_dots:
    p1.circle(x,z,fill_color='white', color=c, size=8)


def gen_hist( p1, data, c, lbl ):
  data = trajectories[T]['data']
  binsz = settings['binsz']
  z = data['elev']
  a = np.arange(z.min(), z.max(), binsz)
  h, edges = np.histogram( data['elev'], range=(z.min(),z.max()), bins=a.size)
  i = np.where( h == h.max())[0]
  i = i[0]
  edges[i]
  to_z = edges[i]
  ppk_zhist( p1, edges[0:-1], h, c, lbl )

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:

  #print(h.max()," Values found at ", to_z, "m Elevation (m) using ", binsz*100.0, "cm bins.")
  #print("This is the most likely takeoff location.")
  display_all_trj_stats()
  p1 = open_new_plot()
  Elevation_histograms_Title =  "2019-0326 DFC" #@param "ddd" {type:"string"}
  binSize_In_cm = 2.5 #@param {type:"slider", min:1, max:100, step:0.5}
  binsz = binSize_In_cm / 100.0
  settings['binsz'] = binsz
  
  if Elevation_histograms_Title == "":
    p1.title.text =  "Historgrams with "+str(binSize_In_cm)+"cm bins"
  else:
    p1.title.text = Elevation_histograms_Title


  p1.xaxis.axis_label = "Elevation (meters)"
  p1.yaxis.axis_label = "Count"
  for T in Trj:
    if T in trajectories and Trj[T] == True and 'data' in trajectories[T]:
      #print(T, c[T], trajectories[T]['ifn'])
      gen_hist( p1, trajectories[T], c[T], str(T) )

  show(p1)


In [0]:
#@title **Plot** Elevation Differences: Trj1-Trj[n]
#@markdown Y axis limits (m)
dif_range = 0.5 #@param {type:"slider", min:0.25, max:2, step:0.25}

Elevation_Plot_Title = "Elevation Differences:" #@param {type:"string"}
Reference_Trajectory = 1 #@param [1,2,3,4,5,6,7] {type:"raw"}


import bokeh.plotting as bkp
import bokeh.models as bkm
try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  p0 = open_time_plot()
  p0.title.text = Elevation_Plot_Title
  p0.yaxis.axis_label = "Elevation Difference (m)"
  print("The Reference is # ", Reference_Trajectory)

  def plot_difs( p, n, c ):
    mn1=trajectories[Reference_Trajectory]['data']['hms_z'].min() 
    mx1=trajectories[Reference_Trajectory]['data']['hms_z'].max()
    td1=mx1-mn1
    mn2=trajectories[n]['data']['hms_z'].min() 
    mx2=trajectories[n]['data']['hms_z'].max()
    td2=mx2-mn2

    if mn1 < mn2:
      common_start = mn2
    else:
      common_start = mn1

    if mx1 > mx2:
      common_stop = mx2
    else:
      common_stop = mx1

    td1,td2,mn1,mn2,mx1,mx2, common_start, common_stop
    y_interp = np.interp(
        trajectories[Reference_Trajectory]['data']['hms_z'],
        trajectories[n]['data']['hms_z'],
        trajectories[n]['data']['elev']
        )
    y_diff = trajectories[Reference_Trajectory]['data']['elev']-y_interp

    y_diff

    p0.line(                          # Plot the elevations vs time
        pd.to_datetime(trajectories[Reference_Trajectory]['data']['hms_z']),
        y_diff,
        color=c,
        legend=str(Reference_Trajectory)+" - "+str(n),
        )
    p0.y_range=bkm.Range1d(-dif_range-.1,dif_range+.1)
    return

  display_all_trj_stats()
  if Reference_Trajectory in trajectories:
    for T in trajectories:
      if T in trajectories and Trj[T] == True and 'data' in trajectories[T] and 'data' in trajectories[Reference_Trajectory]:
        #print("in ",T, c[T], trajectories[T]['ifn'], trajectories[T]['Trajectory_Type'])
        #print(T, c[T],  Trj_fn[T])
        if T != Reference_Trajectory:
          plot_difs( p0, T, c[T] )
    show(p0)
  else:
    print("No reference trajectory selected.")



In [0]:
#@title Plot lat vs Lon

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  Lat_Lon_Title = "Test" #@param {type:"string"}
  display_all_trj_stats()
  p0 = open_new_plot()
  p0.title.text = Elevation_Plot_Title = Lat_Lon_Title
  p0.yaxis.axis_label = "Latitude"
  p0.xaxis.axis_label = "Longitude"
  for T in Trj:
    if T in trajectories and Trj[T] == True and 'data' in trajectories[T]:
      #print("in ",T, c[T], trajectories[T]['ifn'], trajectories[T]['Trajectory_Type'])
      dt =  trajectories[T]['data']
      if plot_lines:
        p0.line(         
          dt['lon'],
          dt['lat'],
          color=c[T],
          legend=str(T),
          line_width=2
          )
      if plot_dots:
        p0.circle_cross(                          # Plot the elevations vs time
          dt['lon'],
          dt['lat'], 
          size=4,
          legend=str(T),
          color=c[T]
          )
    #else:
            #print("else ", T, c[T],Trj[T], Trj_fn[T])
  show(p0)

In [0]:
#@title Plot Stdev95 vs Time
#@markdown Y axis limits (m)
Elev_range = 0.3 #@param {type:"slider", min:0.1, max:5, step:0.1}
import bokeh.plotting as bkp
import bokeh.models as bkm

try:
  init_analysis()
except:
  print("You need to Run the 'Initilize Tool Box' first. Cell at the top." )
else:
  display_all_trj_stats()
  Elevation_Stats = "Elevation Stdev95% & NSATS vs Time " #@param {type:"string"}
  p0 = open_time_plot()
  p0.yaxis.axis_label= "Elevation Std Dev (95%)"
  p0.title.text = Elevation_Stats
  p0.add_layout(bkm.LinearAxis(y_range_name="NSATS"), 'right')
  p0.extra_y_ranges = {"NSATS": bkm.Range1d(start=0, end=20) }
  for T in Trj:
    if T in trajectories and Trj[T] == True and 'data' in trajectories[T]:
      #print(T, c[T], trajectories[T]['ifn'], trajectories[T]['Trajectory_Type'])
      dt =  trajectories[T]['data']
      if plot_dots:
        p0.circle_cross(                         # Plot the elevations vs time
        pd.to_datetime(dt['hms_z']),
        dt['sdu95'], 
        size=4,
        color=c[T]
        )
      if plot_lines:
        p0.line(                                 # Plot the elevations vs time
        pd.to_datetime(dt['hms_z']),
        dt['sdu95'], 
        color=c[T],
        line_width=Line_Width,
        legend="SDU95"
        )
        if 'ns' in dt:
          p0.line(pd.to_datetime(dt['hms_z']),
                dt['ns'],
                color="Black",
                line_width=Line_Width,
                line_dash='dotted',
                legend="NSATS",
                y_range_name="NSATS"
                )
        p0.y_range=bkm.Range1d(0.0,Elev_range+.02)

      p0.y_range=bkm.Range1d(0.0,Elev_range)
  show(p0)

# **D:** Generate Photo Positions.

In [0]:
#@title **D1:** Login, Load the code,  and load the sample data.   {display-mode: "form"}
%%shell

## Upload your ppkuser.privatekey before executing this.
cd /content
if [[ -e ppkuser.private ]]; then
  mkdir -p /root/.ssh                                        #
  cp ppkuser.private /root/.ssh/id_rsa                       #
  chmod 600 /root/.ssh/id_rsa                                #
  ssh-keyscan github.com > /root/.ssh/known_hosts            #
  echo "Downloading the latest ppk module.."
  cd /content                                                #
  rm -rf  *.git                                              #
  git clone git@github.com:lidar532/cwwppkgeotaglib.git      #
  cp cwwppkgeotaglib/cwwppkgeotaglib.py /content             #
  pip install pyproj                                         #
#  echo "Dowwnloading Example PPK Datasets..."
  cd /content
  
#  if [[ -e PPK_Sample_data ]]; then
#    mkdir -p trash
#    mv PPK_Sample_data trash                                 #
#    cd trash                                                 #
#    rm -rf *
#   fi
   
#  cd /content                                                #
#  git clone https://github.com/lidar532/PPK_Sample_data.git  #
  echo "All done, Ready to process data."
else
  echo "No user key file found.  Please upload your ppkuser.private Key File, and try again."
fi

In [0]:
#@title **D2:** Select data files and options, and generate PPK based GeoTags for your photos. {display-mode: "form"}
import os
import sys
import pandas as pd
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

#@markdown #Input Data Files

if IN_COLAB:
  Trajectory_Source      = "RTKlib" #@param ["RTKlib", "PPP-Ca"]
  Trajectory_GPS_to_UTC_Time_difference =  0 #@param {type:"integer" }
  Exif_File_Name         = "/content/2019-0326-DFC-Solo2/01_gps/aircraft/exif/2019-0326-F1F2-exif.txt" #@param {type:"string"}
  Trajectory_file_Name   = "/content/2019-0326-DFC-Solo2/01_gps/trajectories/2019-0326-213228-Solo-GP213208-CTMC-PK12N-cmb-RTKtraj-pos.txt" #@param {type:"string"}
  Flash_Events_file_Name = "/content/2019-0326-DFC-Solo2/01_gps/aircraft/raw/2019-0326-213228-Solo-GP213208.TXT" #@param {type:"string"}

#Exif_File_Name = "/content/0gt/2019-0326-F1F2-exif.txt" #@param {type:"string"}
#Trajectory_file_Name = "/content/0gt/GP213208-U15NG-CTMC.pos" #@param {type:"string"}
#Flash_Events_file_Name = "/content/0gt/GP213208.TXT" #@param {type:"string"}

#@markdown ---
#@markdown #Output File
Base_station_ID = "DFC" #@param {type:"string"}
User_Notes = "3DR Solo Denver Fed Center Solo GR2" #@param {type:"string"}
Generate_Output_File = "Yes" #@param ["No", "Yes"]

#@markdown ---
#@markdown #Stats, Graphs & Plots
Plot_Times = True #@param {type:"boolean" }
Show_File_Stats = True #@param {type:"boolean" }
Show_Flash_event_Distribution = False #@param {type:"boolean" }
Show_XYZ_Std_Devs = True #@param {type:"boolean"} 
Show_Photo_Location_Plan_View = True #@param {type:"boolean"}
Show_Photo_Elevations = True #@param { type: "boolean" }

#@markdown ---
#@markdown #Camera EXIF Time Adjustments
EXIF_drift_correction_seconds =  -7#@param {type:"integer" }
EXIF_Offset_from_UTC_Hours =  6#@param {type:"integer"}

#@markdown ---
Debugging_Output = "None" #@param ["None", "Function EntryExit", ""]
#@markdown ---

try:
    import cwwppkgeotaglib as ppk
    ready = True
except ModuleNotFoundError as err:
    ready = False
    print("The cwwppkgeotaglib library was not found.  You need to run step 1 to load the library first.")
ready = True
#import cwwppkgeotaglib as ppk
if ready:
  print('\nCWW PPK Geotag Library version: ', ppk.ppk['ppk_flash_sync_version'])
  print("Ready to go..")
  ppk.ppk_user_settings['Trajectory_Source']                = Trajectory_Source
  ppk.ppk_user_settings['Exif_File_Name']                   = '/' +  Exif_File_Name
  ppk.ppk_user_settings['Flash_Events_file_Name']           = '/' + Flash_Events_file_Name
  ppk.ppk_user_settings['Trajectory_file_Name']             = '/' + Trajectory_file_Name
  ppk.ppk_user_settings['EXIF_drift_correction_seconds']    = EXIF_drift_correction_seconds
  ppk.ppk_user_settings['EXIF_Offset_from_UTC_Hours']       = EXIF_Offset_from_UTC_Hours
  ppk.ppk_user_settings['Generate_Output_File']             = Generate_Output_File
  ppk.ppk_user_settings['Base_station_ID']                  = Base_station_ID
  ppk.ppk_user_settings['User_Notes']                       = User_Notes
  ppk.ppk_user_settings['Generate_Output_File']             = Generate_Output_File
  ppk.ppk_user_settings['Plot_Times']                       = Plot_Times
  ppk.ppk_user_settings['Show_File_Stats']                  = Show_File_Stats
  ppk.ppk_user_settings['Show_Flash_event_Distribution']    = Show_Flash_event_Distribution
  ppk.ppk_user_settings['Show_XYZ_Std_Devs']                = Show_XYZ_Std_Devs
  ppk.ppk_user_settings['Show_Photo_Location_Plan_View']    = Show_Photo_Location_Plan_View
  ppk.ppk_user_settings['Show_Photo_Elevations']            = Show_Photo_Elevations
  ppk.ppk_user_settings['Debugging_Output']                 = Debugging_Output
  ppk.process_cww_ppk_files( ppk.ppk, ppk.ppk_user_settings)

# F. General Purpose Tools.

In [0]:
#@title F.0.1 Zip a directory for downloading.
#@markdown Paste or type a directory or file below to zip.
zip_name = "/content/2019-0326-DFC-Solo2" #@param {type:"string"}
print('Zipping.  Standby.')
!zip -qr {zip_name} {zip_name}/*
print('Operation Completed.')

Revisions, ToDo List, Wish List, References, Useful Links
===


## To Do List
1. Add numerical statistics to various plots to quantify differences
1. Fix pix pos file namimg duplicates, offset time, stuck H4 issue.
1. Add a designated output directory for pix pos outputs
1. Add in improved tools for examining camera clock issues
1. Translate trajectories to an export trajectory file that the PPK EO tool can read and process.
1. Add histogram for differenced trajectories





## Revisions
* 2019-1216
  * 
  * Consolidated flow and structure
  * Integrated the pix position code with the rest.
* 2019-1214
  * Added ability to extract and use NAD83 coordinates from an OPUS file.
  * Added lat/lon/elevation for base stations input text widget.
* 2019-1211
  * Added linux tools to convert raw data to RINEX
  * Added RTKlib post processing online operations.
* 2019-1109
  * Added cell to delete a selected trajectory from memory.
* 2019-1108
  * Constructed unified statistics display functions
  * fixed various bugs 
  * Added auto-hide for graph menu
  * Added user z_bias so user can offset trajectory elevation if so desired
  * Added additional user input data to help describe each trajectory
*   2019-1106
  * Added stats printout def for read in trajectories
  * Added file names of displayed trjs to printout above each graph.
  * Added User input title to each graph
  * Cleaned up dif plotter
  * Added ability to difference several trj from a reference
  * Preset graph mouse functions for each graphic
  * Improved Trap errors when wrong file loaded, or missing columns
  * Added code to instruct the user to run the initilization cell before using each tool.
*   2019-1005
  * Consolidated the graphic style stuff in a Defs
  * Fixed length difference warnings from some graphs
  * Trapped error when 'ns' column missing from a trajectory in the stats plot
  * Added undo * redo buttons to plots
* 2019-1003



## Wish List
* Add automatic extraction, and creation of GCPs, of UAS takeoff and landing sites.
* Add reader for CWW-PPK_Geotag output file.
* Add reader for John Sontags trajectory files
* Add reader for NASA GIPSE trajectory files
* Add RTKlib compatible output option so we can drive the CWW-PPK processing software
* Add reader and plotting ability for OPUS result files.
* Add generic xy plotter reader, plotter, difference, etc.  For Global Mapper transect plots.
* Add ability to GrafNav reader to find the header row, and use it for column labels
* Add background map to the plan view
* Add a switch for UTM on the plan view
* Add stats to histogram plot
* Add reader for Flash shoe time stamps
* Add plotter for Flash shoe timing display with zoom
* Add reader & plotter for EXIF files.
* Integrate [Juypter ipywidgets](https://ipywidgets.readthedocs.io/en/latest/index.html)


## Useful Links
* [Hexagon SmartNet Global GNSS RINEX data](https://hxgnsmartnet.com/en-us/local-coverage)
* [Florida FPRN GNSS Station map](https://www.myfloridagps.com/map/)
* [Texas CORS Network](http://ftp.dot.state.tx.us/pub/txdot-info/isd/gps/)
* [NOAA CORS Map](https://www.ngs.noaa.gov/CORS_Map/)
* [North Carolina CORS Stations](https://ncgs.state.nc.us/pages/CORS-and-GNSS.htm)
* [UNAVCO global free GPS Data](https://www.unavco.org/instrumentation/networks/status/all)
* [Trimble Correction Services for Survey Applications (Paid Subscription)](https://tpsstore.trimble.com/OA_HTML/tnvopwrdlr_ibeProductGroup.jsp?application=Survey&parentapplication=GEOSPATIAL)
## References
* [All about OPUS](https://outside.vermont.gov/agency/vtrans/external/docs/geodetic/OPUS_2012_MALSCE.pdf)
* [ Satellite DOP Predictor ](http://satpredictor2.deere.com/homePost)
* [Ipython and Shell Commands](https://jakevdp.github.io/PythonDataScienceHandbook/01.05-ipython-and-shell-commands.html)
* [Markdown in Colabs](https://colab.research.google.com/notebooks/markdown_guide.ipynb#scrollTo=70pYkR9LiOV0)
* [ Bokeh interactive visualization library]( https://docs.bokeh.org/en/latest/index.html)
* [Bokeh Interactions, widgets](https://docs.bokeh.org/en/latest/docs/user_guide/interaction.html)
* [Colabs grid widgets](https://colab.research.google.com/notebooks/widgets.ipynb#scrollTo=P6xc9QVFSlrw)
* [os.path for filename manipulation](https://docs.python.org/3/library/os.path.html)
* [Juypter Widget list and examples](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)
* [O'Reilly Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)






---




# G. Developer Stuff.

In [0]:
#@title G.0.1 Download, build, and install RTKlib from lidar.net
#@markdown Downloads RTKlib from lidar.net, 
#@markdown * removes existing binarys
#@markdown * Builds new binaries
#@markdown * Installs then in /usr/local/bin
#@markdown  
#@markdown This cell is intended to be used to generate new binaries
#@markdown when a new version of RTKlib is being configured. 
#@markdown It is not inteneded for users as it takes alot of time to
#@markdown recompile and install the binaries.
!cd /usr/local/bin; rm -v convbin rnx2rtkp pos2kml rtkrcv str2str
!cd /content/; rm -rf RTKLIB
!git clone https://github.com/lidar532/RTKLIB.git
!cd /content/RTKLIB/app/; make clean; make all; make install
!cd /usr/local/bin/; ls -la rnx2rtkp convbin pos2kml rtkrcv str2str
!convbin  -? 2> convbin.txt
!rnx2rtkp -? 2> rnx3rtkp.txt
!pos2kml  -? 2> pos2kml.txt
!rtkrcv   -? 2> rtkrcv.txt 
!str2str  -? 2> str2str.txt
!chmod uog+x /usr/local/bin/rnx2rtkp

!echo "RTKlib is ready for use."

In [0]:
#@title G.0.2. Reload the library after making changes.
#@markdown Use to cause the cwwppkgeotaglib.py to be reloaded so any changes can be used.
from importlib import reload
reload(ppk)

In [0]:
import cwwppkgeotaglib 